# Model Evaluation & Selection

<img src='images/justice-icon.jpg' id="logo" height="75%" width="75%"/>

## Objective

This module is going to introduce you to...

1. Cross-validation procedures for more robust model performance assessment.
2. Various ways to apply different evaluation metrics.
3. Hyperparameter tuning to find optimal model parameter settings.

## Quick refresher

But first, let's review a few things that we learned in the previous modules.

### Data prep

In [36]:
# packages used
import pandas as pd
from sklearn.model_selection import train_test_split

# import data
adult_census = pd.read_csv('../data/adult-census.csv')

# separate feature & target data
target = adult_census['class']
features = adult_census.drop(columns='class')

# drop the duplicated column `"education-num"` as stated in the data exploration notebook
features = features.drop(columns='education-num')

# split into train & test sets
X_train, X_test, y_train, y_test = train_test_split(features, target, random_state=123)

### Feature engineering

In [37]:
# packages used
from sklearn.compose import make_column_selector as selector
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# create selector object based on data type
numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)

# get columns of interest
numerical_columns = numerical_columns_selector(features)
categorical_columns = categorical_columns_selector(features)

# preprocessors to handle numeric and categorical features
numerical_preprocessor = StandardScaler()
categorical_preprocessor = OneHotEncoder(handle_unknown="ignore")

# transformer to associate each of these preprocessors with their respective columns
preprocessor = ColumnTransformer([
    ('one-hot-encoder', categorical_preprocessor, categorical_columns),
    ('standard_scaler', numerical_preprocessor, numerical_columns)
])

### Modeling

In [38]:
# packages used
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

# Pipeline object to chain together modeling processes
model = make_pipeline(preprocessor, LogisticRegression(max_iter=500))
model

# fit our model
_ = model.fit(X_train, y_train)

# score on test set
model.score(X_test, y_test)

0.8503808041929408

## Resampling & cross-validation

In our "03-first_model.ipynb" notebook we split our data into training and testing sets and we assessed the performance of our model on the test set. Unfortunately, there are a few pitfalls to this approach:

1. If our dataset is small, a single test set may not provide realistic expectations of our model's performance on unseen data.
2. A single test set does not provide us any insight on variability of our model's performance.
3. Using our test set to drive our model building process can bias our results via _data leakage_.

Resampling methods provide an alternative approach by allowing us to repeatedly fit a model of interest to parts of the training data and test its performance on other parts of the training data.

<img src='images/resampling.svg' id="logo" height="75%" width="75%"/>

<div class="admonition note alert alert-info">
    <p class="first admonition-title" style="font-weight: bold;"><b>Note</b></p>
<p class="last">This allows us to train and validate our model entirely on the training data and not touch the test data until we have selected a final "optimal" model.</p>
</div>

The two most commonly used resampling methods include ___k-fold cross-validation___ and ___bootstrap sampling___. This module focuses on using k-fold cross-validation.

## K-fold cross-validation

Cross-validation consists of repeating the procedure such that the training and testing sets are different each time. Generalization performance metrics are collected for each repetition and then aggregated. As a result we can get an estimate of the variability of the model’s generalization performance.

_k_-fold cross-validation (aka _k_-fold CV) is a resampling method that randomly divides the training data into _k_ groups (aka folds) of approximately equal size. 

![](images/cross_validation_diagram.png)

The model is fit on $k-1$ folds and then the remaining fold is used to compute model performance.  This procedure is repeated _k_ times; each time, a different fold is treated as the validation set. 

This process results in _k_ estimates of the generalization error (say $\epsilon_1, \epsilon_2, \dots, \epsilon_k$). Thus, the _k_-fold CV estimate is computed by averaging the _k_ test errors, providing us with an approximation of the error we might expect on unseen data.

![](images/cv.png)

In scikit-learn, the function [`cross_validate`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html) allows us to perform cross-validation and you need to pass it the model, the data, and the target. Since there exists several cross-validation strategies, cross_validate takes a parameter `cv` which defines the splitting strategy.

<div class="admonition tip alert alert-warning">
    <p class="first admonition-title" style="font-weight: bold;"><b>Tip</b></p>
    <p class="last">In practice, one typically uses k=5 or k=10. There is no formal rule as to the size of k; however, as k gets larger, the difference between the estimated performance and the true performance to be seen on the test set will decrease.</p>
</ul>
</div>

In [39]:
%%time
from sklearn.model_selection import cross_validate

cv_result = cross_validate(model, X_train, y_train, cv=5)
cv_result

CPU times: user 3.25 s, sys: 56.5 ms, total: 3.3 s
Wall time: 3.31 s


{'fit_time': array([0.61690283, 0.63014293, 0.60851097, 0.66170192, 0.60711718]),
 'score_time': array([0.0261631 , 0.02799892, 0.02674103, 0.02812505, 0.02827287]),
 'test_score': array([0.85191757, 0.84548185, 0.85790336, 0.85094185, 0.85558286])}

The output of cross_validate is a Python dictionary, which by default contains three entries: 

- `fit_time`: the time to train the model on the training data for each fold, 
- `score_time`: the time to predict with the model on the testing data for each fold, and 
- `test_score`: the default score on the testing data for each fold.

In [40]:
scores = cv_result["test_score"]
print("The mean cross-validation accuracy is: "
      f"{scores.mean():.3f} +/- {scores.std():.3f}")

The mean cross-validation accuracy is: 0.852 +/- 0.004


<div class="admonition tip alert alert-warning">
    <p class="first admonition-title" style="font-weight: bold;"><b>Your Turn</b></p>
<p class="last">
    Using <tt class="docutils literal">KNeighborsClassifier()</tt>, run a 5 fold cross validation procedure and compare the accuracy and standard deviation.  <b>Note</b>: Don't forget to create a new model pipeline object.
</p>
</div>

## Evaluation metrics

* Evaluation metrics allow us to measure the predictive accuracy of our model – the difference between the predicted value ($\hat{y}_i$) and the actual value ($y_i$).

* We often refer to evaluation metrics as ___loss functions___: $f(y_{i} - \hat{y}_i)$

* Scikit-Learn provides multiple ways to compute evaluation metrics and refers to this concept as [___scoring___](https://scikit-learn.org/stable/modules/model_evaluation.html). 
   1. Estimator scoring method
   2. Individual scoring functions
   3. Scoring parameters

### Estimator scoring method

Every estimator (regression/classification model) has a default scoring method

Most classifiers return the mean accuracy of the model on the supplied $X$ and $y$:

In [41]:
# toy data
from sklearn.datasets import load_breast_cancer
X_cancer, y_cancer = load_breast_cancer(return_X_y=True)

# fit model
clf = LogisticRegression(solver='liblinear').fit(X_cancer, y_cancer)

# score 
clf.score(X_cancer, y_cancer)

0.9595782073813708

While most regressors return the $R^2$ metric:

In [42]:
# toy data
from sklearn.datasets import load_boston
X_boston, y_boston = load_boston(return_X_y=True)

# fit model
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(X_boston, y_boston)

# score
reg.score(X_boston, y_boston)

0.7406426641094095

### Individual scoring functions

However, these default evaluation metrics are often not the metrics most suitable to the business problem.

There are many loss functions to choose from; each with unique characteristics that can be beneficial for certain problems.
   * Regression problems
      - Mean squared error (MSE)
      - Root mean squared error (RMSE)
      - Mean absolute error (MAE)
      - etc.
   * Classification problems
      - Area under the curve (AUC)
      - Cross-entropy (aka Log loss)
      - Precision
      - etc.

Scikit-Learn provides many [scoring functions](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics) to choose from.

In [43]:
from sklearn import metrics

The functions take actual y values and predicted y values -- $f(y_{i} - \hat{y}_i)$

__Example regression metrics__:

In [44]:
y_pred = reg.predict(X_boston)

# Mean squared error
metrics.mean_squared_error(y_boston, y_pred)

21.894831181729202

In [45]:
# Mean absolute percentage error
metrics.mean_absolute_percentage_error(y_boston, y_pred)

0.1641729880648995

__Example classification metrics__:

In [46]:
y_pred = clf.predict(X_cancer)

# Area under the curve
metrics.roc_auc_score(y_cancer, y_pred)

0.9543760900586651

In [47]:
# F1 score
metrics.f1_score(y_cancer, y_pred)

0.968011126564673

In [48]:
# multiple metrics at once!
print(metrics.classification_report(y_cancer, y_pred))

              precision    recall  f1-score   support

           0       0.96      0.93      0.95       212
           1       0.96      0.97      0.97       357

    accuracy                           0.96       569
   macro avg       0.96      0.95      0.96       569
weighted avg       0.96      0.96      0.96       569



### Scoring parameters

And since we prefer to use cross-validation procedures, scikit-learn has incorporated a `scoring` parameter.

Most evaluation metrics have a predefined text string that can be supplied as a `scoring` argument. 

In [49]:
# say we wanted to use AUC as our loss function while using 5-fold validation
cross_validate(model, X_train, y_train, cv=5, scoring='roc_auc')

{'fit_time': array([0.70541596, 0.71939898, 0.69874835, 0.96958804, 0.72835779]),
 'score_time': array([0.031564  , 0.03213   , 0.03276587, 0.04343081, 0.03146601]),
 'test_score': array([0.90485472, 0.90326931, 0.91316978, 0.90553186, 0.90816178])}

<div class="admonition note alert alert-info">
    <p class="first admonition-title" style="font-weight: bold;"><b>Note</b></p>
    <p class="last">The unified scoring API in scikit-learn always <u>maximizes</u> the score, so metrics which need to be minimized are negated in order for the unified scoring API to work correctly. Consequently, some metrics such as <tt class="docutils literal">mean_squared_error()</tt> will use a predefined text string starting with <b>neg_</b> (i.e. 'neg_mean_squared_error').</p>
</div>

In [50]:
# applying mean squared error in a regression k-fold cross validation procedure
cross_validate(reg, X_boston, y_boston, cv=5, scoring='neg_root_mean_squared_error')

{'fit_time': array([0.00123978, 0.00123715, 0.00064993, 0.00054502, 0.00053811]),
 'score_time': array([0.000458  , 0.00037909, 0.00023985, 0.00023079, 0.00022507]),
 'test_score': array([-3.52991509, -5.10378498, -5.75101191, -8.9867887 , -5.77179405])}

You can even supply [more than one metric](https://scikit-learn.org/stable/modules/model_evaluation.html#using-multiple-metric-evaluation) or even [define your own custom metric](https://scikit-learn.org/stable/modules/model_evaluation.html#defining-your-scoring-strategy-from-metric-functions).

In [51]:
# example of supplying more than one metric
metrics = ['accuracy', 'roc_auc']

cross_validate(model, X_train, y_train, cv=5, scoring=metrics)

{'fit_time': array([0.67276406, 0.66832423, 0.61833882, 0.63187504, 0.59966493]),
 'score_time': array([0.05943894, 0.06058502, 0.05659795, 0.05282521, 0.05487514]),
 'test_accuracy': array([0.85191757, 0.84548185, 0.85790336, 0.85094185, 0.85558286]),
 'test_roc_auc': array([0.90485472, 0.90326931, 0.91316978, 0.90553186, 0.90816178])}

<div class="admonition tip alert alert-warning">
    <p class="first admonition-title" style="font-weight: bold;"><b>Your Turn</b></p>
<p class="last">
Using the <tt class="docutils literal">KNeighborsClassifier()</tt> from the previous <b>Your Turn</b> exercises, perform a 5 fold cross validation and compute the accuracy <i><u>and</u></i> ROC AUC.
</p>
</div>

## Hyperparameter tuning

Given two different models (blue line) to the same data (gray dots), which model do you prefer?

A | B
- | - 
![alt](images/modeling-process-bias-model-1.png) | ![alt](images/modeling-process-variance-model-1.png)

Prediction errors can be decomposed into two main subcomponents we care about:

- error due to “bias”
- error due to “variance”

### Bias

Error due to ___bias___ is the difference between the expected (or average) prediction of our model and the correct value which we are trying to predict.

It measures how far off in general a model’s predictions are from the correct value, which provides a sense of how well a model can conform to the underlying structure of the data.

High bias models (i.e. generalized linear models) are rarely affected by the noise introduced by new unseen data

![](images/modeling-process-bias-model-2.png)

### Variance

Error due to ___variance___ is the variability of a model prediction for a given data point.

Many models (e.g., k-nearest neighbor, decision trees, gradient boosting machines) are very adaptable and offer extreme flexibility in the patterns that they can fit to. However, these models offer their own problems as they run the risk of overfitting to the training data. 

Although you may achieve very good performance on your training data, the model will not automatically generalize well to unseen data.

![](images/modeling-process-variance-model-2.png)

<div class="admonition note alert alert-info">
    <p class="first admonition-title" style="font-weight: bold;"><b>Note</b></p>
<p class="last">Many high performing models (i.e. random forests, gradient boosting machines, deep learning) are very flexible in the patterns they can conform to due to the many hyperparameters they have. However, this also means they are prone to overfitting (aka can have high variance error).</p>
</div>

___Hyperparameters___ (aka tuning parameters) are the "knobs to twiddle" to control the complexity of machine learning algorithms and, therefore, the ___bias-variance trade-off___.

Some models have very few hyperparameters. For example in a K-nearest neighbor (KNN) model ___K___ (the number of neighbors) is the primary hyperparameter.

![](images/modeling-process-knn-options-1.png)

While other models such as gradient boosted machines (GBMs) and deep learning models can have many.

___Hyperparameter tuning___ is the process of screening hyperparameter values (or combinations of hyperparameter values) to find a model that balances bias & variance so that the model generalizes well to unseen data.

In [52]:
%%time
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

# set hyperparameter in KNN model 
model = KNeighborsClassifier(n_neighbors=10)

# create preprocessor & modeling pipeline
pipeline = make_pipeline(preprocessor, model)

# 5-fold cross validation using AUC error metric
results = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='roc_auc')

f'KNN model with 10 neighbors: AUC = {np.mean(results):.3f}'

CPU times: user 48.3 s, sys: 8.77 s, total: 57 s
Wall time: 57.3 s


'KNN model with 10 neighbors: AUC = 0.883'

But what if we wanted to assess and compare `n_neighbors` = 5, 10, 15, 20, ... ?

For this we could use a ___full cartesian grid search___ using Scikit-Learn's `GridSearchCV()`:

In [53]:
%%time
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

# basic model object
knn = KNeighborsClassifier()

# Create grid of hyperparameter values
hyper_grid = {'knn__n_neighbors': [5, 10, 15, 20]}

# create preprocessor & modeling pipeline
pipeline = Pipeline([('prep', preprocessor), ('knn', knn)])

# Tune a knn model using grid search
grid_search = GridSearchCV(pipeline, hyper_grid, cv=5, scoring='roc_auc', n_jobs=-1)
results = grid_search.fit(X_train, y_train)

# Best model's cross validated AUC
abs(results.best_score_)

CPU times: user 577 ms, sys: 466 ms, total: 1.04 s
Wall time: 2min 53s


0.8936630646924394

<div class="admonition tip alert alert-warning">
    <p class="first admonition-title" style="font-weight: bold;"><b>Tip</b></p>
    <p class="last">We use <tt class="docutils literal">Pipeline</tt> rather than <tt class="docutils literal">make_pipeline</tt> in the above because it allows us to name the different steps in the pipeline. This allows us to assign hyperparameters to distinct steps within the pipeline.</p>
</div>

In [54]:
results.best_params_

{'knn__n_neighbors': 20}

However, a cartesian grid-search approach has limitations. 

* It does not scale well when the number of parameters to tune is increasing.
* It also forces regularity rather than aligning values assessed to distributions.

![](images/random_search.png)

<div class="admonition note alert alert-info">
    <p class="first admonition-title" style="font-weight: bold;"><b>Note</b></p>
    <p class="last">Random search based on hyperparameter distributions has proven to perform as well, if not better than, standard grid search. Learn more <a href="http://jmlr.csail.mit.edu/papers/volume13/bergstra12a/bergstra12a.pdf">here</a>.</p>
</div>

For example, say we want to train a random forest classifier. Random forests are very flexible algorithms and can have _several_ hyperparameters. 

In [66]:
from sklearn.ensemble import RandomForestClassifier

# basic model object
rf = RandomForestClassifier(random_state=123)

# create preprocessor & modeling pipeline
pipeline = Pipeline([('prep', preprocessor), ('rf', rf)])

For this particular random forest algorithm we'll assess the following hyperparameters. Don't worry if you are not familiar with what these do.

* `n_estimators`: number of trees in the forest,
* `max_features`: number of features to consider when looking for the best split,
* `max_depth`: maximum depth of each tree built,
* `min_samples_leaf`: minimum number of samples required in a leaf node,
* `max_samples`: number of samples to draw from our training data to train each tree.

A standard grid search would be very computationally intense.

Instead, we'll use a random latin hypercube search using [`RandomizedSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html).

To build our grid, we need to specify distributions for our hyperparameters.

<div class="admonition tip alert alert-warning">
    <p class="first admonition-title" style="font-weight: bold;"><b>Tip</b></p>
    <p class="last"><tt class="docutils literal">scipy.stats.loguniform</tt> can be used to generate floating numbers. To generate random values for integer-valued parameters (e.g. <tt class="docutils literal">min_samples_leaf</tt>) we can adapt is as follows:</p>
</div>

In [75]:
from scipy.stats import loguniform


class loguniform_int:
    """Integer valued version of the log-uniform distribution"""
    def __init__(self, a, b):
        self._distribution = loguniform(a, b)

    def rvs(self, *args, **kwargs):
        """Random variable sample"""
        return self._distribution.rvs(*args, **kwargs).astype(int)

In [76]:
# specify hyperparameter distributions to randomly sample from
param_distributions = {
    'rf__n_estimators': loguniform_int(50, 1000),
    'rf__max_features': loguniform(.1, .5),
    'rf__max_depth': loguniform_int(4, 20),
    'rf__min_samples_leaf': loguniform_int(1, 100),
    'rf__max_samples': loguniform(.5, 1),
}

Now, we can define the randomized search using the different distributions. 

Executing 10 iterations of 5-fold cross-validation for random parametrizations of this model on this dataset can take from 10 seconds to several minutes, depending on the speed of the host computer and the number of available processors.

In [80]:
%%time
from sklearn.model_selection import RandomizedSearchCV

# perform 10 random iterations
random_search = RandomizedSearchCV(
    pipeline, 
    param_distributions=param_distributions, 
    n_iter=10,
    cv=5, 
    scoring='roc_auc',
    verbose=1,
    n_jobs=-1,
)

results = random_search.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
CPU times: user 19.2 s, sys: 351 ms, total: 19.5 s
Wall time: 3min 53s


In [81]:
results.best_score_

0.9167993399776069

In [82]:
results.best_params_

{'rf__max_depth': 19,
 'rf__max_features': 0.4440733300753526,
 'rf__max_samples': 0.9487528482967953,
 'rf__min_samples_leaf': 4,
 'rf__n_estimators': 66}

## Wrapping up

This module discussed:

1. How to use `cross_validate` to perform _k_-fold cross-validation procedures.
2. The three different ways to apply different evaluation metrics:
   a. Default estimator score method - `estimator.score()`
   b. Individual scoring functions - `from sklearn import metrics`
   c. Scoring parameters - `cross_validate(..., scoring='roc_auc')`
3. How to use `GridSearchCV` and `RandomizedSearchCV` to perform hyperparameter tuning.